In [2]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure,show
from bokeh.io import output_notebook
output_notebook()
from bokeh.models.annotations import Span
from bokeh.models.annotations import BoxAnnotation
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource
from bokeh.models import Range1d

from datetime import datetime as dt
from bokeh.io import export_png

Loading BokehJS ...

In [3]:
df_19_15 = pd.read_csv("../data/311_data_19_15_filtered.csv")
df_12_19 = pd.read_csv("../data/311_data_12_19_filtered.csv")

In [5]:
df_12_15 = pd.concat([df_12_19, df_19_15])
df_12_15.head()

,index,Case Reference,Open Date,Closed Date,Status,Subject,Reason,Type,Object Type,Address Number,...,GEOID20_block,Property_Address,Print Key,Address,Zipcode_y,filter_id,GEOID20_blockgroup_y,type,category,Geography
0,0,1001753435,2022-12-13,12/15/2022 08:52:00 AM,Closed,Dept of Public Works,Animal Shelter,Animals (Req_Serv),Property,539,...,360290065011000,100.21-3-34 539 ASHLAND,100.21-3-34,539 ASHLAND,14222.0,210 ONE FAMILY DWELLING,360290065011,Animals (Req_Serv),Issues related to animals and pests,360290065011
1,1,1001753891,2022-12-15,12/20/2022 11:50:00 AM,Closed,Dept of Public Works,Animal Shelter,Animals (Req_Serv),Property,539,...,360290065011000,100.21-3-34 539 ASHLAND,100.21-3-34,539 ASHLAND,14222.0,210 ONE FAMILY DWELLING,360290065011,Animals (Req_Serv),Issues related to animals and pests,360290065011
2,2,1001754233,2022-12-18,12/20/2022 12:20:00 PM,Closed,Dept of Public Works,Streets,Street Snow Plowing (Req_Serv),Property,32,...,360290065011002,100.21-5-31.21 32 GILL,100.21-5-31.21,32 GILL,14222.0,210 ONE FAMILY DWELLING,360290065011,Street Snow Plowing (Req_Serv),Issues related to snow,360290065011
3,3,1001753823,2022-12-15,12/17/2022 07:56:00 AM,Closed,Dept of Public Works,Sanitation,Garbage Missed Pick Up (Req_Serv),Property,485,...,360290065011002,100.21-5-36 485 NORWOOD,100.21-5-36,485 NORWOOD,14222.0,210 ONE FAMILY DWELLING,360290065011,Garbage Missed Pick Up (Req_Serv),Issues related to garbage/recycling collection,360290065011
4,4,1001754086,2022-12-16,12/16/2022 12:50:00 PM,Closed,Dept of Parking,Parking Violations Bureau,Parking Issues (Req_Serv),Property,799,...,360290169001004,100.22-2-87 799 ELMWOOD,100.22-2-87,799 ELMWOOD,14222.0,482 DOWNTOWN ROW TYPE (DETACHED),360290169001,Parking Issues (Req_Serv),Issues related to roads and parking,360290169001


### Overall temporal pattern

In [6]:
groupby_time = df_12_15.groupby(["Open Date"]).agg({"Case Reference":"count"}).reset_index()

groupby_time['Open Date'] = groupby_time['Open Date'].astype(str)
groupby_time['open_date'] = groupby_time['Open Date'].apply(lambda x: str(x).split("-", 1)[1].replace("-","/"))

lis_x = ['12/12','12/13','12/14','12/15','12/16','12/17','12/18','12/19','12/20','12/21','12/22','12/23','12/24','12/25','12/26','12/27','12/28','12/29','12/30','12/31','01/01','01/02','01/03','01/04','01/05','01/06','01/07','01/08','01/09','01/10','01/11','01/12','01/13','01/14','01/15']

date_index = groupby_time['open_date'].copy()

In [8]:
p = figure(height=350, width=1200, x_range=lis_x)

blackout = BoxAnnotation(left=14.5, right=34.5, fill_alpha=0.1, fill_color='limegreen',line_color='limegreen', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout = BoxAnnotation(left=11.5, right=14.5, fill_alpha=0.1, fill_color='orangered',line_color='orangered', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout1 = BoxAnnotation(left=7.5, right=11.5, fill_alpha=0.13, fill_color='gold',line_color='gold', line_width=2, line_dash=[4,4])
p.add_layout(blackout1)

source = ColumnDataSource(data={
    'date' : groupby_time['open_date'],
    'count' : groupby_time['Case Reference']
    })

p.line(x='date', y='count', source=source, width = 7, color="tomato", alpha=1)
p.circle(x='date', y='count', source=source, size=15, line_color="white", fill_color="tomato", line_width=2, alpha=1)

p.xaxis.axis_label_text_font_size = "20pt"
p.xaxis.axis_label_text_font = "Arial"
p.xaxis.major_label_orientation = "vertical"
p.xaxis.axis_label_text_font_style = "bold"
p.xaxis.axis_line_width = 1
p.xaxis.major_label_text_font_size = "20pt"
p.xaxis.major_label_text_font = "Arial"

p.yaxis.axis_label_text_font_size = "20pt"
p.yaxis.axis_label_text_font = "Arial"
p.yaxis.axis_line_width = 1
p.yaxis.major_label_text_font_size = "20pt"
p.yaxis.major_label_text_font = "Arial"

p.axis.minor_tick_out = 6
p.axis.major_tick_out = 10

show(p)

### Requests related to snow removal

In [9]:
requests = df_12_15[(df_12_15['category'] == "Issues related to snow")].copy()
groupby_time = requests.groupby(["Open Date"]).agg({"Case Reference":"count"}).reset_index()

groupby_time['Open Date'] = groupby_time['Open Date'].astype(str)
groupby_time['open_date'] = groupby_time['Open Date'].apply(lambda x: str(x).split("-", 1)[1].replace("-","/"))

groupby_time1 = pd.merge(groupby_time, date_index, left_on="open_date", right_on="open_date", how = "right")
groupby_time1.fillna(0, inplace=True)

lis_x = ['12/12','12/13','12/14','12/15','12/16','12/17','12/18','12/19','12/20','12/21','12/22','12/23','12/24','12/25','12/26','12/27','12/28','12/29','12/30','12/31','01/01','01/02','01/03','01/04','01/05','01/06','01/07','01/08','01/09','01/10','01/11','01/12','01/13','01/14','01/15']

In [10]:
p = figure(height=300, width=900, x_range=lis_x)

source = ColumnDataSource(data={
    'date' : groupby_time1['open_date'],
    'count' : groupby_time1['Case Reference']
    })

p.line(x='date', y='count', source=source, width = 7, color="slateblue", alpha=1)
p.circle(x='date', y='count', source=source, size=15, line_color="white", fill_color="slateblue", line_width=2, alpha=1)

p.xaxis.axis_label_text_font_size = "20pt"
p.xaxis.axis_label_text_font = "Arial"
p.xaxis.major_label_orientation = "vertical"
p.xaxis.axis_label_text_font_style = "bold"
p.xaxis.axis_line_width = 1
p.xaxis.major_label_text_font_size = "20pt"
p.xaxis.major_label_text_font = "Arial"

p.yaxis.axis_label_text_font_size = "20pt"
p.yaxis.axis_label_text_font = "Arial"
p.yaxis.axis_line_width = 1
p.yaxis.major_label_text_font_size = "20pt"
p.yaxis.major_label_text_font = "Arial"

p.axis.minor_tick_out = 6
p.axis.major_tick_out = 10

blackout = BoxAnnotation(left=14.5, right=34.5, fill_alpha=0.1, fill_color='limegreen',line_color='limegreen', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout = BoxAnnotation(left=11.5, right=14.5, fill_alpha=0.1, fill_color='orangered',line_color='orangered', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout1 = BoxAnnotation(left=7.5, right=11.5, fill_alpha=0.13, fill_color='gold',line_color='gold', line_width=2, line_dash=[4,4])
p.add_layout(blackout1)

show(p)

### Requests related to garbage/recycling collection

In [11]:
requests = df_12_15[(df_12_15['category'] == "Issues related to garbage/recycling collection")].copy()
groupby_time = requests.groupby(["Open Date"]).agg({"Case Reference":"count"}).reset_index()

groupby_time['Open Date'] = groupby_time['Open Date'].astype(str)
groupby_time['open_date'] = groupby_time['Open Date'].apply(lambda x: str(x).split("-", 1)[1].replace("-","/"))

groupby_time1 = pd.merge(groupby_time, date_index, left_on="open_date", right_on="open_date", how = "right")
groupby_time1.fillna(0, inplace=True)

lis_x = ['12/12','12/13','12/14','12/15','12/16','12/17','12/18','12/19','12/20','12/21','12/22','12/23','12/24','12/25','12/26','12/27','12/28','12/29','12/30','12/31','01/01','01/02','01/03','01/04','01/05','01/06','01/07','01/08','01/09','01/10','01/11','01/12','01/13','01/14','01/15']

In [13]:
p = figure(height=300, width=900, x_range=lis_x)

source = ColumnDataSource(data={
    'date' : groupby_time1['open_date'],
    'count' : groupby_time1['Case Reference']
    })

p.line(x='date', y='count', source=source, width = 7, color="darkorange", alpha=1)
p.circle(x='date', y='count', source=source, size=15, line_color="white", fill_color="darkorange", line_width=2, alpha=1)

p.xaxis.axis_label_text_font_size = "20pt"
p.xaxis.axis_label_text_font = "Arial"
p.xaxis.major_label_orientation = "vertical"
p.xaxis.axis_label_text_font_style = "bold"
p.xaxis.axis_line_width = 1
p.xaxis.major_label_text_font_size = "20pt"
p.xaxis.major_label_text_font = "Arial"

p.yaxis.axis_label_text_font_size = "20pt"
p.yaxis.axis_label_text_font = "Arial"
p.yaxis.axis_line_width = 1
p.yaxis.major_label_text_font_size = "20pt"
p.yaxis.major_label_text_font = "Arial"

p.axis.minor_tick_out = 6
p.axis.major_tick_out = 10

blackout = BoxAnnotation(left=14.5, right=34.5, fill_alpha=0.1, fill_color='limegreen',line_color='limegreen', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout = BoxAnnotation(left=11.5, right=14.5, fill_alpha=0.1, fill_color='orangered',line_color='orangered', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout1 = BoxAnnotation(left=7.5, right=11.5, fill_alpha=0.13, fill_color='gold',line_color='gold', line_width=2, line_dash=[4,4])
p.add_layout(blackout1)

show(p)

### Requests related to roads and parking

In [16]:
requests = df_12_15[(df_12_15['category'] == "Issues related to roads and parking")].copy()
groupby_time = requests.groupby(["Open Date"]).agg({"Case Reference":"count"}).reset_index()

groupby_time['Open Date'] = groupby_time['Open Date'].astype(str)
groupby_time['open_date'] = groupby_time['Open Date'].apply(lambda x: str(x).split("-", 1)[1].replace("-","/"))

groupby_time1 = pd.merge(groupby_time, date_index, left_on="open_date", right_on="open_date", how = "right")
groupby_time1.fillna(0, inplace=True)

lis_x = ['12/12','12/13','12/14','12/15','12/16','12/17','12/18','12/19','12/20','12/21','12/22','12/23','12/24','12/25','12/26','12/27','12/28','12/29','12/30','12/31','01/01','01/02','01/03','01/04','01/05','01/06','01/07','01/08','01/09','01/10','01/11','01/12','01/13','01/14','01/15']

In [17]:
p = figure(height=300, width=900, x_range=lis_x)

source = ColumnDataSource(data={
    'date' : groupby_time1['open_date'],
    'count' : groupby_time1['Case Reference']
    })

p.line(x='date', y='count', source=source, width = 7, color="grey", alpha=1)
p.circle(x='date', y='count', source=source, size=15, line_color="white", fill_color="grey", line_width=2, alpha=1)

p.xaxis.axis_label_text_font_size = "20pt"
p.xaxis.axis_label_text_font = "Arial"
p.xaxis.major_label_orientation = "vertical"
p.xaxis.axis_label_text_font_style = "bold"
p.xaxis.axis_line_width = 1
p.xaxis.major_label_text_font_size = "20pt"
p.xaxis.major_label_text_font = "Arial"

p.yaxis.axis_label_text_font_size = "20pt"
p.yaxis.axis_label_text_font = "Arial"
p.yaxis.axis_line_width = 1
p.yaxis.major_label_text_font_size = "20pt"
p.yaxis.major_label_text_font = "Arial"

p.axis.minor_tick_out = 6
p.axis.major_tick_out = 10

blackout = BoxAnnotation(left=14.5, right=34.5, fill_alpha=0.1, fill_color='limegreen',line_color='limegreen', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout = BoxAnnotation(left=11.5, right=14.5, fill_alpha=0.1, fill_color='orangered',line_color='orangered', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout1 = BoxAnnotation(left=7.5, right=11.5, fill_alpha=0.13, fill_color='gold',line_color='gold', line_width=2, line_dash=[4,4])
p.add_layout(blackout1)

show(p)

### Requests related to trees

In [19]:
requests = df_12_15[(df_12_15['category'] == "Issues related to trees")].copy()
groupby_time = requests.groupby(["Open Date"]).agg({"Case Reference":"count"}).reset_index()

groupby_time['Open Date'] = groupby_time['Open Date'].astype(str)
groupby_time['open_date'] = groupby_time['Open Date'].apply(lambda x: str(x).split("-", 1)[1].replace("-","/"))

groupby_time1 = pd.merge(groupby_time, date_index, left_on="open_date", right_on="open_date", how = "right")
groupby_time1.fillna(0, inplace=True)

lis_x = ['12/12','12/13','12/14','12/15','12/16','12/17','12/18','12/19','12/20','12/21','12/22','12/23','12/24','12/25','12/26','12/27','12/28','12/29','12/30','12/31','01/01','01/02','01/03','01/04','01/05','01/06','01/07','01/08','01/09','01/10','01/11','01/12','01/13','01/14','01/15']

In [20]:
p = figure(height=300, width=900, x_range=lis_x)

source = ColumnDataSource(data={
    'date' : groupby_time1['open_date'],
    'count' : groupby_time1['Case Reference']
    })

p.line(x='date', y='count', source=source, width = 7, color="forestgreen", alpha=1)
p.circle(x='date', y='count', source=source, size=15, line_color="white", fill_color="forestgreen", line_width=2, alpha=1)

p.xaxis.axis_label_text_font_size = "20pt"
p.xaxis.axis_label_text_font = "Arial"
p.xaxis.major_label_orientation = "vertical"
p.xaxis.axis_label_text_font_style = "bold"
p.xaxis.axis_line_width = 1
p.xaxis.major_label_text_font_size = "20pt"
p.xaxis.major_label_text_font = "Arial"

p.yaxis.axis_label_text_font_size = "20pt"
p.yaxis.axis_label_text_font = "Arial"
p.yaxis.axis_line_width = 1
p.yaxis.major_label_text_font_size = "20pt"
p.yaxis.major_label_text_font = "Arial"

p.axis.minor_tick_out = 6
p.axis.major_tick_out = 10

blackout = BoxAnnotation(left=14.5, right=34.5, fill_alpha=0.1, fill_color='limegreen',line_color='limegreen', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout = BoxAnnotation(left=11.5, right=14.5, fill_alpha=0.1, fill_color='orangered',line_color='orangered', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout1 = BoxAnnotation(left=7.5, right=11.5, fill_alpha=0.13, fill_color='gold',line_color='gold', line_width=2, line_dash=[4,4])
p.add_layout(blackout1)

show(p)

### Requests related to housing

In [21]:
requests = df_12_15[(df_12_15['category'] == "Issues related to housing")].copy()
groupby_time = requests.groupby(["Open Date"]).agg({"Case Reference":"count"}).reset_index()

groupby_time['Open Date'] = groupby_time['Open Date'].astype(str)
groupby_time['open_date'] = groupby_time['Open Date'].apply(lambda x: str(x).split("-", 1)[1].replace("-","/"))

groupby_time1 = pd.merge(groupby_time, date_index, left_on="open_date", right_on="open_date", how = "right")
groupby_time1.fillna(0, inplace=True)

lis_x = ['12/12','12/13','12/14','12/15','12/16','12/17','12/18','12/19','12/20','12/21','12/22','12/23','12/24','12/25','12/26','12/27','12/28','12/29','12/30','12/31','01/01','01/02','01/03','01/04','01/05','01/06','01/07','01/08','01/09','01/10','01/11','01/12','01/13','01/14','01/15']

In [22]:
p = figure(height=300, width=900, x_range=lis_x)

source = ColumnDataSource(data={
    'date' : groupby_time1['open_date'],
    'count' : groupby_time1['Case Reference']
    })

p.line(x='date', y='count', source=source, width = 7, color="darkkhaki", alpha=1)
p.circle(x='date', y='count', source=source, size=15, line_color="white", fill_color="darkkhaki", line_width=2, alpha=1)

p.xaxis.axis_label_text_font_size = "20pt"
p.xaxis.axis_label_text_font = "Arial"
p.xaxis.major_label_orientation = "vertical"
p.xaxis.axis_label_text_font_style = "bold"
p.xaxis.axis_line_width = 1
p.xaxis.major_label_text_font_size = "20pt"
p.xaxis.major_label_text_font = "Arial"

p.yaxis.axis_label_text_font_size = "20pt"
p.yaxis.axis_label_text_font = "Arial"
p.yaxis.axis_line_width = 1
p.yaxis.major_label_text_font_size = "20pt"
p.yaxis.major_label_text_font = "Arial"

p.axis.minor_tick_out = 6
p.axis.major_tick_out = 10

blackout = BoxAnnotation(left=14.5, right=34.5, fill_alpha=0.1, fill_color='limegreen',line_color='limegreen', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout = BoxAnnotation(left=11.5, right=14.5, fill_alpha=0.1, fill_color='orangered',line_color='orangered', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout1 = BoxAnnotation(left=7.5, right=11.5, fill_alpha=0.13, fill_color='gold',line_color='gold', line_width=2, line_dash=[4,4])
p.add_layout(blackout1)

show(p)

### Requests relted to water and sewer

In [24]:
requests = df_12_15[(df_12_15['category'] == "Issues related to water and sewer")].copy()
groupby_time = requests.groupby(["Open Date"]).agg({"Case Reference":"count"}).reset_index()

groupby_time['Open Date'] = groupby_time['Open Date'].astype(str)
groupby_time['open_date'] = groupby_time['Open Date'].apply(lambda x: str(x).split("-", 1)[1].replace("-","/"))

groupby_time1 = pd.merge(groupby_time, date_index, left_on="open_date", right_on="open_date", how = "right")
groupby_time1.fillna(0, inplace=True)

lis_x = ['12/12','12/13','12/14','12/15','12/16','12/17','12/18','12/19','12/20','12/21','12/22','12/23','12/24','12/25','12/26','12/27','12/28','12/29','12/30','12/31','01/01','01/02','01/03','01/04','01/05','01/06','01/07','01/08','01/09','01/10','01/11','01/12','01/13','01/14','01/15']

In [25]:
p = figure(height=300, width=900, x_range=lis_x)

source = ColumnDataSource(data={
    'date' : groupby_time1['open_date'],
    'count' : groupby_time1['Case Reference']
    })

p.line(x='date', y='count', source=source, width = 7, color="teal", alpha=1)
p.circle(x='date', y='count', source=source, size=15, line_color="white", fill_color="teal", line_width=2, alpha=1)

p.xaxis.axis_label_text_font_size = "20pt"
p.xaxis.axis_label_text_font = "Arial"
p.xaxis.major_label_orientation = "vertical"
p.xaxis.axis_label_text_font_style = "bold"
p.xaxis.axis_line_width = 1
p.xaxis.major_label_text_font_size = "20pt"
p.xaxis.major_label_text_font = "Arial"

p.yaxis.axis_label_text_font_size = "20pt"
p.yaxis.axis_label_text_font = "Arial"
p.yaxis.axis_line_width = 1
p.yaxis.major_label_text_font_size = "20pt"
p.yaxis.major_label_text_font = "Arial"

p.axis.minor_tick_out = 6
p.axis.major_tick_out = 10

blackout = BoxAnnotation(left=14.5, right=34.5, fill_alpha=0.1, fill_color='limegreen',line_color='limegreen', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout = BoxAnnotation(left=11.5, right=14.5, fill_alpha=0.1, fill_color='orangered',line_color='orangered', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout1 = BoxAnnotation(left=7.5, right=11.5, fill_alpha=0.13, fill_color='gold',line_color='gold', line_width=2, line_dash=[4,4])
p.add_layout(blackout1)

show(p)

### Requests related to animals and pests

In [26]:
requests = df_12_15[(df_12_15['category'] == "Issues related to animals and pests")].copy()
groupby_time = requests.groupby(["Open Date"]).agg({"Case Reference":"count"}).reset_index()

groupby_time['Open Date'] = groupby_time['Open Date'].astype(str)
groupby_time['open_date'] = groupby_time['Open Date'].apply(lambda x: str(x).split("-", 1)[1].replace("-","/"))

groupby_time1 = pd.merge(groupby_time, date_index, left_on="open_date", right_on="open_date", how = "right")
groupby_time1.fillna(0, inplace=True)

lis_x = ['12/12','12/13','12/14','12/15','12/16','12/17','12/18','12/19','12/20','12/21','12/22','12/23','12/24','12/25','12/26','12/27','12/28','12/29','12/30','12/31','01/01','01/02','01/03','01/04','01/05','01/06','01/07','01/08','01/09','01/10','01/11','01/12','01/13','01/14','01/15']

In [27]:
p = figure(height=300, width=900, x_range=lis_x)

source = ColumnDataSource(data={
    'date' : groupby_time1['open_date'],
    'count' : groupby_time1['Case Reference']
    })

p.line(x='date', y='count', source=source, width = 7, color="fuchsia", alpha=1)
p.circle(x='date', y='count', source=source, size=15, line_color="white", fill_color="fuchsia", line_width=2, alpha=1)

p.xaxis.axis_label_text_font_size = "20pt"
p.xaxis.axis_label_text_font = "Arial"
p.xaxis.major_label_orientation = "vertical"
p.xaxis.axis_label_text_font_style = "bold"
p.xaxis.axis_line_width = 1
p.xaxis.major_label_text_font_size = "20pt"
p.xaxis.major_label_text_font = "Arial"

p.yaxis.axis_label_text_font_size = "20pt"
p.yaxis.axis_label_text_font = "Arial"
p.yaxis.axis_line_width = 1
p.yaxis.major_label_text_font_size = "20pt"
p.yaxis.major_label_text_font = "Arial"

p.axis.minor_tick_out = 6
p.axis.major_tick_out = 10

blackout = BoxAnnotation(left=14.5, right=34.5, fill_alpha=0.1, fill_color='limegreen',line_color='limegreen', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout = BoxAnnotation(left=11.5, right=14.5, fill_alpha=0.1, fill_color='orangered',line_color='orangered', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout1 = BoxAnnotation(left=7.5, right=11.5, fill_alpha=0.13, fill_color='gold',line_color='gold', line_width=2, line_dash=[4,4])
p.add_layout(blackout1)

show(p)

### Other requests

In [28]:
requests = df_12_15[(df_12_15['category'] == "Other issues")].copy()
groupby_time = requests.groupby(["Open Date"]).agg({"Case Reference":"count"}).reset_index()

groupby_time['Open Date'] = groupby_time['Open Date'].astype(str)
groupby_time['open_date'] = groupby_time['Open Date'].apply(lambda x: str(x).split("-", 1)[1].replace("-","/"))

groupby_time1 = pd.merge(groupby_time, date_index, left_on="open_date", right_on="open_date", how = "right")
groupby_time1.fillna(0, inplace=True)

lis_x = ['12/12','12/13','12/14','12/15','12/16','12/17','12/18','12/19','12/20','12/21','12/22','12/23','12/24','12/25','12/26','12/27','12/28','12/29','12/30','12/31','01/01','01/02','01/03','01/04','01/05','01/06','01/07','01/08','01/09','01/10','01/11','01/12','01/13','01/14','01/15']

In [29]:
p = figure(height=300, width=900, x_range=lis_x)

source = ColumnDataSource(data={
    'date' : groupby_time1['open_date'],
    'count' : groupby_time1['Case Reference']
    })

p.line(x='date', y='count', source=source, width = 7, color="darkorchid", alpha=1)
p.circle(x='date', y='count', source=source, size=15, line_color="white", fill_color="darkorchid", line_width=2, alpha=1)

p.xaxis.axis_label_text_font_size = "20pt"
p.xaxis.axis_label_text_font = "Arial"
p.xaxis.major_label_orientation = "vertical"
p.xaxis.axis_label_text_font_style = "bold"
p.xaxis.axis_line_width = 1
p.xaxis.major_label_text_font_size = "20pt"
p.xaxis.major_label_text_font = "Arial"

p.yaxis.axis_label_text_font_size = "20pt"
p.yaxis.axis_label_text_font = "Arial"
p.yaxis.axis_line_width = 1
p.yaxis.major_label_text_font_size = "20pt"
p.yaxis.major_label_text_font = "Arial"

p.axis.minor_tick_out = 6
p.axis.major_tick_out = 10

blackout = BoxAnnotation(left=14.5, right=34.5, fill_alpha=0.1, fill_color='limegreen',line_color='limegreen', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout = BoxAnnotation(left=11.5, right=14.5, fill_alpha=0.1, fill_color='orangered',line_color='orangered', line_width=2, line_dash=[4,4])
p.add_layout(blackout)
blackout1 = BoxAnnotation(left=7.5, right=11.5, fill_alpha=0.13, fill_color='gold',line_color='gold', line_width=2, line_dash=[4,4])
p.add_layout(blackout1)

show(p)